### <가치함수 Q를 추정하기 위한 살사(활성 정책 TD 제어)> 
출처 : 단단한 머신러닝 챕터 6 

알고리즘 파라미터 : 시간 간격 $\alpha \in (0,1] $, 작은 양수 $\epsilon > 0$ 

모든 $s \in S^+$ 에 대한 Q(s,a)를 임의의 값으로 초기화. 단, Q(종단, -) = 0 

각 에피소드에 대한 루프 : 
- S를 초기화 
- Q(ex- 입실론 탐욕적)로부터 유도된 정책을 사용하여 S로부터 A를 선택 
- 에피소드의 각 단계에 대한 루프 : 

> 행동 A를 취하고 $R,S'$ 을 관측 

> Q로부터 유도된 정책을 사용하여 S'으로부터 A'를 선택 

> Q(s,a) ← $Q(s,a) + \alpha[R+ \gamma Q(s',a') - Q(s,a)] $

> S ← S', A ← A' 

S가 종단이면 종료 


**<구현해야 하는 것>**
- Q(a,s) :  

**<필요한 것>** 
- $\alpha$ : class 제작시 입력 값으로 부여 
- $\epsilon$ : 충분히 작은 값으로 

**<함수 / 데이터 형식>** 
- class evaluate_TD : # alpha 값 추가 
> def __init__(self, S, A, alpha reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 


**<외부함수>** 
- R(s',a,s) : 보상함수. 챕터 4의 코드 참고 
- choose_random_max(lst) : lst 중 가장 값이 큰 것을 반환. 혹시 max 값이 중복된다면 임의의 하나 산출


**<고민점>** 
- 어떻게 Q(s,a)를 초기화하는 것이 좋을까? 
- Q(s,a)의 데이터 형식을 어떻게 잡는 것이 좋을까? 

> 이중 리스트, dict 둘다 가능. 이번에는 Q(s',a')를 불러와야 하는 과정이 있으니, 속도 향상을 위해 dict 적용 




In [1]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [2]:
from collections import defaultdict
import random
import numpy as np

In [3]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 
    
#최대값이 2개 이상인 경우, 임의로 1개의 최대값을 만들어낸 행동 a를 산출 
def choose_random_max(lst) :
    max_arg = np.where(np.array(lst) >= max(lst))
    return random.choice(list(max_arg)[0]) #max_arg가 array 형태로 안에 있는 list를 꺼내기 위해 [0] 사용 



In [9]:
# TD(0) 예측 코드 참고 

class evaluate_Q_TD :  
    def __init__(self, S, A, reward_func, alpha=0.1,  epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.reward_func = reward_func
        self.alpha = alpha
        self.epsilon = epsilon 
        self.gamma = gamma 
        self.num_episode = num_episode
        self.T = len_episode
        self.Q = self.initiate_Q()
#        self.b, self.C = self.initiate_b()
#        self.pi = pi 
#        self.V = self.initiate_V() 
    
    def initiate_Q(self) : # Q(s,a) 값을 초기화 
        Q_dict = defaultdict(float)
        for s in self.S : 
            for a in self.A : 
                Q_dict[(s,a)] = 0         
        
        return Q_dict
    
    def pi(self, s) : #Q 값을 기반해서 가장 가치가 높은 행동 a 산출 
        lst = [] 
        for a in self.A : lst.append(self.Q[(s,a)])
        return choose_random_max(lst)
    

    def choice_action(self, s, policy) : #일반화. 정책 기반으로 상황 s에 있을 때 선택할 행동 a 산출 
        policy_a_list = [] 
        for _ in self.A :
            policy_a_list.append(policy(s,_)) 

        a= random.choices(self.A, weights = policy_a_list)
        a = a[0]
        return a
    
    def next_s(self, s,a) : # 상태 s에서 a 행동을 했을 때 다음 상태 s'. 정책, S,A 에 따라 달라짐. 
        return min(max(s+a, 0), max(self.S)) 

    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a = self.pi(s)  # pi 함수가 결정론적으로 a 값을 반환함에 따라 수정 
            next_s = self.next_s(s, a)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # Q 추정 및 제어를 위해 수정 
        
        for _ in range(self.num_episode) : 
            start_s = random.choice(self.S) # 시작 탐험 가정 
            episode = self.make_episode(start_s, self.T) 
            S,R,A = episode['S'],episode['R'], episode['A'] 
            # make_episode 에서 이미 a,r를 계산해 두었기 때문에 Q(s,a)만 갱신하겠음. 
            for index, s in enumerate(S[:-1]) :  
                index_origin_s = self.S.index(s)
                next_s = S[index+1]
                self.Q[(s, R[index])] = self.Q[(s,R[index])] + self.alpha*(R[index+1] + self.gamma*self.Q[(next_s, A[index+1])] - self.Q[(s, R[index])]) 
        

In [12]:
test = evaluate_Q_TD(S,A, reward_func)
lst = [] 

for _ in range(1000) : 
    lst.append(list(test.Q.values())[:])
    test.update_returns()

print(lst[0])
print(lst[-1])

# 경우의 수가 너무 많아선가, 최적의 정책으로 수렴하는 모습이 잘 안보임. 

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

[0, 0, 0, 0, 0, -0.11073405382539683, 0, 0, 0, 0, 0, 0, 0, 0, -0.1, -0.24906029615806335, 0, 0, 0, 0, 0, 0, 0, 0, -0.16818571428571427, -0.09647524682539682, 0, 0, 0, 0, 0, 0, 0, 0, -0.037311111111111114, -0.10085688480819699, 0, 0, 0, 0, 0, 0, 0, 0, 0.04, -0.01400792857142856, 0, 0, 0, 0, 0, 0, 0, 0, -0.03333333333333333, -0.010648603463516882, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.09714573754283673, 0, 0, 0, 0, 0, 0, 0, 0, -0.018000000000000002, -0.10244113343741898, 0, 0, 0, 0, 0, 0, 0, 0, 0.07, 0.05862040596201602, 0, 0, 0, 0, 0, 0, 0, 0, -0.051333333333333335, -0.1464489407877143, 0, 0, 0, 0, 0, 0, 0, 0, 0.02725, -0.016352069600000005, 0, 0, 0, 0, 0, 0, 0, 0, -0.11000000000000001, -0.3493477326126359, 0, 0, 0, 0, 0, 0, 0, 0, 0.11082850352764204, 0.09336049936470203, 0, 0, 0, 0, 0, 0, 0, 0, -0.011111111111111112, 0.0006052795227488098, 0, 0, 0, 0, 0, 0, 0, 0, 0.020050000000000002, 0.012641500351800217, 0, 0, 0, 0, 0, 0, 0, 0, 0.025, 0.03174446724999999, 0, 0, 0, 0, 0, 0, 0, 0, 0, -0.060387